In [6]:
# SymPy derivation for the 2p_x transition dipole:
# - starts from ψ̃_{2p_x}(p,θ,φ) in spherical momentum coords
# - applies the spherical gradient operator
# - rotates to Cartesian basis
# - removes all trig using identities to express the result purely in (px, py, pz)
# NOTE: no output/printing; results are stored in variables.

import sympy as sp

# Symbols
p, theta, phi, gamma = sp.symbols('p theta phi gamma', positive=True, real=True)
px, py, pz = sp.symbols('px py pz', real=True)
N = sp.symbols('N', real=True)  # e.g., N = 8*sqrt(2)*gamma**(sp.Rational(7,2))/pi if desired

# Spherical unit vectors (as Cartesian components)
e_r = sp.Matrix([sp.sin(theta)*sp.cos(phi),
                 sp.sin(theta)*sp.sin(phi),
                 sp.cos(theta)])
e_theta = sp.Matrix([sp.cos(theta)*sp.cos(phi),
                     sp.cos(theta)*sp.sin(phi),
                     -sp.sin(theta)])
e_phi = sp.Matrix([-sp.sin(phi), sp.cos(phi), 0])

# Spherical→Cartesian rotation matrix (columns are e_r, e_theta, e_phi)
R = sp.Matrix.hstack(e_r, e_theta, e_phi)

# Real momentum-space 2p_x orbital: ψ̃_{2p_x}(p,θ,φ)
psi_tilde_2px = N * p * sp.sin(theta) * sp.cos(phi) / (gamma**2 + p**2)**3

# Spherical gradient components of ψ̃ (v_r, v_theta, v_phi):
df_dp     = sp.diff(psi_tilde_2px, p)
df_dtheta = sp.diff(psi_tilde_2px, theta)
df_dphi   = sp.diff(psi_tilde_2px, phi)

v_r     = df_dp
v_theta = df_dtheta / p
v_phi   = df_dphi   / (p * sp.sin(theta))

v_sph = sp.Matrix([v_r, v_theta, v_phi])

# Rotate spherical components into Cartesian basis (still in terms of p,θ,φ)
dipole_2px_cart_angles = sp.simplify(R * v_sph)

# Substitute angles → Cartesian momentum components and remove residual trig
p_norm = sp.sqrt(px**2 + py**2 + pz**2)

subs_rules = {
    # basic relations
    p: p_norm,
    sp.sin(theta)*sp.cos(phi): px/p_norm,
    sp.sin(theta)*sp.sin(phi): py/p_norm,
    sp.cos(theta):             pz/p_norm,
    sp.sin(theta)**2:          (px**2 + py**2)/p_norm**2,

    # identities that appear after rotation (no square-roots needed)
    sp.sin(phi - 2*theta) - sp.sin(phi + 2*theta): -4*px*pz/p_norm**2,
    sp.cos(phi - 2*theta) - sp.cos(phi + 2*theta):  4*py*pz/p_norm**2,
}

dipole_2px_cart = sp.simplify(sp.together(dipole_2px_cart_angles.subs(subs_rules)))

# Optional: factor/organize (kept as separate variables, still no output)
dipole_2px_cart_factored = sp.simplify(sp.factor(dipole_2px_cart))
dipole_2px_cart_components = [sp.simplify(sp.together(c)) for c in dipole_2px_cart_factored]

# Optional: in-plane specialization (pz = 0), if needed later
dipole_2px_inplane = [sp.simplify(c.subs({pz: 0})) for c in dipole_2px_cart_components]

display(dipole_2px_cart_factored)

Matrix([
[N*(gamma**2 - 5*px**2 + py**2 + pz**2)/(gamma**2 + px**2 + py**2 + pz**2)**4],
[                            -6*N*px*py/(gamma**2 + px**2 + py**2 + pz**2)**4],
[                            -6*N*px*pz/(gamma**2 + px**2 + py**2 + pz**2)**4]])